In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
import copy
import datetime
import os
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [3]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
command, segment, block, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)
sar = celeri.process_sar(sar, command)
segment, station, block, sar = celeri.assign_block_labels(segment, station, block, sar)

Reached SAR data


In [4]:
geodetic_data = celeri.merge_geodetic_data(station, sar)

In [5]:
# celeri.plot_block_labels(segment, block, station)

In [6]:
sar.columns

Index(['lon', 'lat', 'line_of_sight_change_val', 'line_of_sight_change_sig',
       'look_vector_x', 'look_vector_y', 'look_vector_z', 'reference_point_x',
       'reference_point_y', 'dep', 'x', 'y', 'z', 'block_label'],
      dtype='object')

In [7]:
sar.columns

Index(['lon', 'lat', 'line_of_sight_change_val', 'line_of_sight_change_sig',
       'look_vector_x', 'look_vector_y', 'look_vector_z', 'reference_point_x',
       'reference_point_y', 'dep', 'x', 'y', 'z', 'block_label'],
      dtype='object')